## 加载数据

In [2]:
import skfeature.utility.util

from feature_selection import FeatureSelection

# 数据集参数
Gisette = {'DATASETNAME': 'gisette.mat'}
Isolet = {'DATASETNAME': 'Isolet.mat'}
Madelon = {'DATASETNAME': 'madelon.mat'}
Wine = {'DATASETNAME': 'Wine.mat'}
Datasets = [Gisette, Isolet, Madelon, Wine]

fs = FeatureSelection(Datasets)
fs.pre_process(Datasets[3], random_state=42)
fs.display_distribution()

trainset distribution: [41 50 33]
testset distribution: [18 21 15]
number of feature: 13


In [ ]:
import numpy as np

def convert_ranking(arr):
    n = len(arr)
    arr_revers = np.zeros(n, dtype=int)
    for rank in range(n):
        feature_index = arr[rank]
        arr_revers[feature_index] = rank + 1  # 假设排名从1开始
    return arr_revers

## 特征选择
- 应用不同的特征选择算法

In [9]:
from skfeature.function.similarity_based import fisher_score
from skfeature.function.statistical_based import CFS
from skfeature.function.information_theoretical_based import FCBF,CIFE
from skfeature.function.wrapper import decision_tree_backward

# idx_11 = fisher_score.fisher_score(fs.x_train, fs.y_train)
# print(idx_11)
# idx_12 = fisher_score.fisher_score(fs.x_train, fs.y_train, mode='index')
# print(idx_12)
# idx_21 = decision_tree_backward.decision_tree_backward(fs.x_train, fs.y_train,n_selected_features=50)
# print(idx_21)
# idx_22 = decision_tree_backward.decision_tree_backward(fs.x_train, fs.y_train, mode='index',n_selected_features=50)
# print(idx_22)
idx_31 = CIFE.cife(fs.x_train, fs.y_train,n_selected_features=13)
print(idx_31)
idx_32 = CIFE.cife(fs.x_train, fs.y_train, mode='index',n_selected_features=13)
print(idx_32)

[ 6 12  5  8  9 10  2  4  7 11  0  1  3]
[ 6  0  7  4  3  2 10  8  5  1 12 11  9]


## 非支配排序

In [ ]:
import numpy as np


def non_dominated_sort(arr1, arr2, arr3):
    """
    对三个特征排名数组进行非支配排序，返回所有前沿
    
    参数:
        arr1, arr2, arr3: 一维ndarray数组，表示三个不同的特征排名
        
    返回:
        list of lists: 每个子列表代表一个前沿面，按Front 1, Front 2, ...排列
    """
    # 1. 合并三个数组的排名
    num_features = len(arr1)
    features = np.column_stack((arr1, arr2, arr3))

    # 2. 非支配排序
    domination_counts = np.zeros(num_features, dtype=int)
    dominated_features = [[] for _ in range(num_features)]
    fronts = [[]]  # fronts[0] = Front 1, fronts[1] = Front 2, ...

    # 计算支配关系
    for i in range(num_features):
        for j in range(num_features):
            if i == j:
                continue
            # 检查i是否支配j
            if np.all(features[i] <= features[j]) and np.any(features[i] < features[j]):
                dominated_features[i].append(j)
                domination_counts[j] += 1

    # 初始化第一前沿面（Front 1）
    fronts[0] = [i for i in range(num_features) if domination_counts[i] == 0]

    # 构建后续前沿面
    current_front = 0
    while fronts[current_front]:
        next_front = []
        for i in fronts[current_front]:
            for j in dominated_features[i]:
                domination_counts[j] -= 1
                if domination_counts[j] == 0:
                    next_front.append(j)
        current_front += 1
        if next_front:
            fronts.append(next_front)

    # 3. 返回所有前沿（每个前沿内的特征按原始索引升序排列）
    return [sorted(front) for front in fronts]


# 示例用法
if __name__ == "__main__":
    # 示例数据
    arr1 = np.array([1, 3, 2, 5, 4])  # 算法1的排名（越小越好）
    arr2 = np.array([2, 1, 3, 4, 5])  # 算法2的排名
    arr3 = np.array([1, 2, 4, 3, 5])  # 算法3的排名

    all_fronts = non_dominated_sort(arr1, arr2, arr3)
    print("所有Pareto前沿：")
    for i, front in enumerate(all_fronts, 1):
        print(f"Front {i}: {front}")

In [ ]:
import numpy as np
from skfeature.utility.util import reverse_argsort
idx = np.argsort(np.array([1.1, 2.2, 0.8, 3.3, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]), 0)
print(idx[::-1])
print(reverse_argsort(idx[::-1],idx.size ))

In [14]:
import numpy as np

def convert_ranking(arr):
    n = len(arr)
    arr_revers = np.zeros(n, dtype=int)
    for rank in range(n):
        feature_index = arr[rank]
        arr_revers[feature_index] = rank + 1  # 假设排名从1开始
    return arr_revers

# 示例
arr = np.array([2, 4,0, 1, 3])
# 3 4 1 5 2
arr_revers = convert_ranking(arr)
print(arr_revers)

[3 4 1 5 2]
